In [1]:
using Pkg
Pkg.instantiate()
import TAMode
Pkg.add("Turing")
using Turing

┌ Info: Precompiling TAMode [294f26d0-a9c2-4f6a-959f-92a060cb6778]
└ @ Base loading.jl:1273


  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/TAMode.jl/Project.toml`
 [no changes]
  Updating `~/TAMode.jl/Manifest.toml`
 [no changes]


In [2]:
tpoints = 10.0 .^ range(-6.0, stop = 2.0, length = 10)

@model test(tps, ::Type{TV}=Vector{Float64}) where {TV} = begin
    params ~ MvNormal(fill(0.5, 15), 0.01)

    data = TAMode.getAutocrine(params)
    #data = TAMode.runTAM(tps, params, 10.0)
end

(::DynamicPPL.ModelGen{(:tps, :TV),var"###test#431",NamedTuple{(:TV,),Tuple{Type{Array{Float64,1}}}}}) (generic function with 1 method)

In [3]:
samp = sample(test(tpoints), HMC(0.1, 5), 100)

AssertionError: AssertionError: all(params .>= 0.0)